In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os, gc, glob
import itertools
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')

In [2]:
# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16 or not. feather format does not support float16.
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            # skip datetime type or categorical type
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
subfiles = glob.glob('/kaggle/input/*/submission.csv')

subfiles

['/kaggle/input/ashrae-catboost-infer/submission.csv',
 '/kaggle/input/ashrae-catboostiii-prediction/submission.csv',
 '/kaggle/input/ashrae-kfold-lightgbm/submission.csv',
 '/kaggle/input/ashrae-2-lightgbm-without-leak-data/submission.csv',
 '/kaggle/input/ashrae-lgb-drop/submission.csv',
 '/kaggle/input/ashrae-no-folds/submission.csv',
 '/kaggle/input/ashrae-stratified-kfold/submission.csv',
 '/kaggle/input/ashrae-gkfold-lgb/submission.csv',
 '/kaggle/input/ashrae-embednn/submission.csv']

In [4]:
len(subfiles)

9

In [5]:
subs = pd.read_csv(subfiles[0], index_col=0, dtype={'meter_reading':'float32'})

subs = subs.rename(columns={'meter_reading':'p0'})
subs['p0'] = subs['p0'].map(np.log1p)
subs = reduce_mem_usage(subs)
for i in tqdm(range(1, 9)):
    sub = pd.read_csv(subfiles[i], index_col=0, dtype={'meter_reading':'float32'})
    sub['meter_reading'] = sub['meter_reading'].map(np.log1p)
    sub = reduce_mem_usage(sub)
    subs['p%s'%i] = sub['meter_reading']
    del sub
    gc.collect()

Memory usage of dataframe is 636.25 MB


  0%|          | 0/8 [00:00<?, ?it/s]

Memory usage after optimization is: 477.19 MB
Decreased by 25.0%
Memory usage of dataframe is 636.25 MB


 12%|█▎        | 1/8 [01:35<11:07, 95.37s/it]

Memory usage after optimization is: 477.19 MB
Decreased by 25.0%
Memory usage of dataframe is 636.25 MB


...

 75%|███████▌  | 6/8 [09:36<03:12, 96.14s/it]

Memory usage after optimization is: 477.19 MB
Decreased by 25.0%
Memory usage of dataframe is 636.25 MB


 88%|████████▊ | 7/8 [11:14<01:36, 96.58s/it]

Memory usage after optimization is: 477.19 MB
Decreased by 25.0%
Memory usage of dataframe is 636.25 MB
Memory usage after optimization is: 477.19 MB
Decreased by 25.0%


100%|██████████| 8/8 [12:48<00:00, 96.04s/it]


In [6]:
subs.reset_index(drop=True, inplace=True)

subs.to_feather('subs.feather')

del subs
gc.collect()

22

In [7]:
leak_df = pd.read_feather('../input/ashrae-leak-data-station/leak.feather')
leak_df.fillna(0, inplace=True)
leak_df = leak_df[(leak_df.timestamp.dt.year > 2016) & (leak_df.timestamp.dt.year < 2019)]
leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0 # remove large negative values
leak_df = leak_df[leak_df.building_id!=245]
leak_df = leak_df.sort_values(['timestamp', 'building_id', 'meter_reading']).drop_duplicates(['building_id', 'meter', 'timestamp'], keep='last')
leak_df = reduce_mem_usage(leak_df)

Memory usage of dataframe is 460.05 MB
Memory usage after optimization is: 299.03 MB
Decreased by 35.0%


In [8]:
test = pd.read_csv('/kaggle/input/ashrae-energy-prediction/test.csv', parse_dates=['timestamp'], 
                   dtype={'meter':'int8', 'building_id':'int32'})

leak_df = leak_df.merge(test, on=['building_id', 'meter', 'timestamp'], how='left')

leak_df[['row_id', 'meter_reading']].to_csv('leak.csv', index=False)